# mill-py Quick Start

This notebook demonstrates connecting to a Mill data service, running queries,
and consuming results as Python dicts, Arrow Tables, pandas DataFrames, and polars DataFrames.

**Prerequisites**:
- A running Mill service (gRPC on port 9090 or HTTP on port 8080)
- `pip install qpointz-mill-py[all]` (or `cd clients && make install`)

## Setup

In [ ]:
# Configuration — adjust to your environment
GRPC_URL = "grpc://localhost:9090"
HTTP_URL = "http://localhost:8080/services/jet"
SCHEMA = "skymill"

## 1. Connect & Handshake

In [ ]:
from mill import connect

client = connect(GRPC_URL)

resp = client.handshake()
print(f"Protocol: {resp.version}")
print(f"SQL support: {resp.capabilities.supportSql}")
print(f"Paging support: {resp.capabilities.supportResultPaging}")

## 2. List Schemas

In [ ]:
schemas = client.list_schemas()
print(f"Available schemas: {schemas}")

## 3. Schema Introspection

In [ ]:
schema = client.get_schema(SCHEMA)

for table in schema.tables:
    print(f"\n{table.schema_name}.{table.name} ({table.table_type.name})")
    for field in table.fields:
        null = "NULL" if field.nullable else "NOT NULL"
        print(f"  {field.name}: {field.type.name} {null}")

## 4. Run a Query — Row Dicts

In [ ]:
sql = f'SELECT "id", "city", "country_code" FROM "{SCHEMA}"."cities" LIMIT 10'
print(f"SQL: {sql}\n")

result = client.query(sql)
for row in result:
    print(row)

## 5. PyArrow Table

In [ ]:
result = client.query(sql)
arrow_table = result.to_arrow()

print(f"Rows: {arrow_table.num_rows}, Columns: {arrow_table.num_columns}")
print(f"Schema:\n{arrow_table.schema}")
arrow_table

## 6. pandas DataFrame

In [ ]:
result = client.query(sql)
df = result.to_pandas()
df

## 7. polars DataFrame

In [ ]:
result = client.query(sql)
pl_df = result.to_polars()
pl_df

## 8. Aggregation Query

In [ ]:
agg_sql = f"""
SELECT "country_code", COUNT(*) AS "city_count"
FROM "{SCHEMA}"."cities"
GROUP BY "country_code"
ORDER BY "city_count" DESC
LIMIT 10
""".strip()

print(f"SQL:\n{agg_sql}\n")
result = client.query(agg_sql)
result.to_pandas()

## 9. HTTP Connection

In [ ]:
# HTTP with JSON encoding (default)
http_client = connect(HTTP_URL)

result = http_client.query(f'SELECT "id", "city" FROM "{SCHEMA}"."cities" LIMIT 5')
result.to_pandas()

In [ ]:
# HTTP with protobuf encoding (more efficient for large results)
proto_client = connect(HTTP_URL, encoding="protobuf")

result = proto_client.query(f'SELECT COUNT(*) AS "total" FROM "{SCHEMA}"."cities"')
result.fetchall()

## 10. Authentication

In [ ]:
from mill.auth import BasicAuth, BearerToken

# Basic auth
# auth_client = connect(GRPC_URL, auth=BasicAuth("reader", "password"))

# Bearer token
# auth_client = connect(GRPC_URL, auth=BearerToken("eyJhbGci..."))

# Anonymous (default — no auth header sent)
anon_client = connect(GRPC_URL)
resp = anon_client.handshake()
print(f"Identity: {resp.authentication.name}")

## 11. Async API

In [ ]:
from mill.aio import connect as aconnect

async_client = await aconnect(GRPC_URL)

schemas = await async_client.list_schemas()
print(f"Schemas (async): {schemas}")

result = await async_client.query(f'SELECT "id", "city" FROM "{SCHEMA}"."cities" LIMIT 5')
df = await result.to_pandas()
df

## 12. Error Handling

In [ ]:
from mill import MillQueryError, MillConnectionError

try:
    client.query("SELECT FROM WHERE INVALID").fetchall()
except MillQueryError as e:
    print(f"Query error: {e}")

try:
    bad = connect("grpc://nonexistent:9090")
    bad.handshake()
except MillConnectionError as e:
    print(f"Connection error: {e}")

## Cleanup

In [ ]:
client.close()
http_client.close()
proto_client.close()
anon_client.close()
await async_client.close()

print("All clients closed.")